In [ ]:
import pandas as pd
import numpy as np
import random
import math

Simplifications: no items, no secondary effects (status, recoil, recharges, stat raise, etc.), 1v1 battles.

## Data

### Main Pokemon data

In [ ]:
all_pokemon = pd.read_csv('data/Pokemon.csv', index_col = 0, usecols =[i for i in range(12)])

all_pokemon = all_pokemon[~(all_pokemon['Name'].str.contains('Mega ', case=False))]

all_pokemon.sample(10)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
#,,,,,,,,,,,
630,Mandibuzz,Dark,Flying,510,110,65,105,55,95,80,5
609,Chandelure,Ghost,Fire,520,60,55,90,145,90,80,5
479,RotomWash Rotom,Electric,Water,520,50,65,107,105,107,86,4
108,Lickitung,Normal,NaN,385,90,55,75,60,75,30,1
594,Alomomola,Water,NaN,470,165,75,80,40,45,65,5
587,Emolga,Electric,Flying,428,55,75,60,75,60,103,5
386,DeoxysNormal Forme,Psychic,NaN,600,50,150,50,150,50,150,3
591,Amoonguss,Grass,Poison,464,114,85,70,85,80,30,5
649,Genesect,Bug,Steel,600,71,120,95,120,95,99,5


### Type Effectiveness

In [106]:
# Define the type matchup data
type_matchups = {
    'Attacking': {
        'Normal': [1, 1, 1, 1, 1, 0.5, 1, 0, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        'Fighting': [2, 1, 0.5, 0.5, 1, 2, 0.5, 0, 2, 1, 1, 1, 1, 0.5, 2, 1, 2, 0.5],
        'Flying': [1, 2, 1, 1, 1, 0.5, 2, 1, 0.5, 1, 1, 2, 0.5, 1, 1, 1, 1, 1],
        'Poison': [1, 1, 1, 0.5, 0.5, 0.5, 1, 0.5, 0, 1, 1, 2, 1, 1, 1, 1, 1, 2],
        'Ground': [1, 1, 0, 2, 1, 2, 0.5, 1, 2, 2, 1, 0.5, 2, 1, 1, 1, 1, 1],
        'Rock': [1, 0.5, 2, 1, 0.5, 1, 2, 1, 0.5, 2, 1, 1, 1, 1, 2, 1, 1, 1],
        'Bug': [1, 0.5, 0.5, 0.5, 1, 1, 1, 0.5, 0.5, 0.5, 1, 2, 1, 2, 1, 1, 2, 0.5],
        'Ghost': [0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0.5, 1],
        'Steel': [1, 1, 1, 1, 1, 2, 1, 1, 0.5, 0.5, 0.5, 1, 0.5, 1, 2, 1, 1, 2],
        'Fire': [1, 1, 1, 1, 1, 0.5, 2, 1, 2, 0.5, 0.5, 2, 1, 1, 2, 0.5, 1, 1],
        'Water': [1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 0.5, 0.5, 1, 1, 1, 0.5, 1, 1],
        'Grass': [1, 1, 0.5, 0.5, 2, 2, 0.5, 1, 0.5, 0.5, 2, 0.5, 1, 1, 1, 0.5, 1, 1],
        'Electric': [1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 2, 0.5, 0.5, 1, 1, 0.5, 1, 1],
        'Psychic': [1, 2, 1, 2, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 0.5, 1, 1, 0, 1],
        'Ice': [1, 1, 2, 1, 2, 1, 1, 1, 0.5, 0.5, 0.5, 2, 1, 1, 0.5, 2, 1, 1],
        'Dragon': [1, 1, 1, 1, 1, 1, 1, 1, 0.5, 1, 1, 1, 1, 1, 1, 2, 1, 0],
        'Dark': [1, 0.5, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 0.5, 0.5],
        'Fairy': [1, 2, 1, 0.5, 1, 1, 1, 1, 0.5, 0.5, 1, 1, 1, 1, 1, 2, 2, 1]
    },
    'Defending': ['Normal', 'Fighting', 'Flying', 'Poison', 'Ground', 'Rock', 'Bug', 'Ghost', 'Steel', 
                  'Fire', 'Water', 'Grass', 'Electric', 'Psychic', 'Ice', 'Dragon', 'Dark', 'Fairy']
}

# Create DataFrame
matchups = pd.DataFrame(type_matchups['Attacking'], index=type_matchups['Defending']).T

# Rename index and columns for clarity
matchups.index.name = "Attacking Type"
matchups.columns.name = "Defending Type"

In [107]:
print("Pokémon Type Effectiveness (Gen VI+):")
print("Rows = Attacking Type, Columns = Defending Type")
matchups

Pokémon Type Effectiveness (Gen VI+):
Rows = Attacking Type, Columns = Defending Type


Defending Type,Normal,Fighting,Flying,Poison,Ground,Rock,Bug,Ghost,Steel,Fire,Water,Grass,Electric,Psychic,Ice,Dragon,Dark,Fairy
Attacking Type,,,,,,,,,,,,,,,,,,
Normal,1.0,1.0,1.0,1.0,1.0,0.5,1.0,0.0,0.5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
Fighting,2.0,1.0,0.5,0.5,1.0,2.0,0.5,0.0,2.0,1.0,1.0,1.0,1.0,0.5,2.0,1.0,2.0,0.5
Flying,1.0,2.0,1.0,1.0,1.0,0.5,2.0,1.0,0.5,1.0,1.0,2.0,0.5,1.0,1.0,1.0,1.0,1.0
Poison,1.0,1.0,1.0,0.5,0.5,0.5,1.0,0.5,0.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0
Ground,1.0,1.0,0.0,2.0,1.0,2.0,0.5,1.0,2.0,2.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,1.0
Rock,1.0,0.5,2.0,1.0,0.5,1.0,2.0,1.0,0.5,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0
Bug,1.0,0.5,0.5,0.5,1.0,1.0,1.0,0.5,0.5,0.5,1.0,2.0,1.0,2.0,1.0,1.0,2.0,0.5
Ghost,0.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,1.0
Steel,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,0.5,0.5,0.5,1.0,0.5,1.0,2.0,1.0,1.0,2.0


### Moves

In [186]:
moves = pd.read_csv('data/metadata_pokemon_moves.csv')
moves.rename(columns=
             {
                 'name': 'Name',
                 'id': 'ID',
                 'accuracy': 'Accuracy',
                 'pp': 'PP',
                 'power': 'Power',
                 'priority': 'Priority',
                 'type': 'Type',
                 'generation': 'Generation',
                 'short_description': 'Short Description',
                 'damage_class': 'Category'
                 },
                   inplace=True)
moves 

,Name,ID,Accuracy,PP,Power,Priority,Type,Generation,short_descripton,Category
0,Pound,1,100.0,35,40.0,0,Normal,Generation I,Inflicts regular damage with no additional eff...,Physical
1,Karate Chop,2,100.0,25,50.0,0,Fighting,Generation I,Has an increased chance for a critical hit.,Physical
2,Double Slap,3,85.0,10,15.0,0,Normal,Generation I,Hits 2-5 times in one turn.,Physical
3,Comet Punch,4,85.0,15,18.0,0,Normal,Generation I,Hits 2-5 times in one turn.,Physical
4,Mega Punch,5,85.0,20,80.0,0,Normal,Generation I,Inflicts regular damage with no additional eff...,Physical
...,...,...,...,...,...,...,...,...,...,...
803,Fiery Wrath,822,100.0,10,90.0,0,Dark,Generation Viii,Inflicts regular damage with no additional eff...,Special
804,Thunderous Kick,823,100.0,10,90.0,0,Fighting,Generation Viii,Inflicts regular damage with no additional eff...,Physical
805,Glacial Lance,824,100.0,5,130.0,0,Ice,Generation Viii,Inflicts regular damage with no additional eff...,Physical
806,Astral Barrage,825,100.0,5,120.0,0,Ghost,Generation Viii,Inflicts regular damage with no additional eff...,Special


### Learnsets

In [104]:
all_learnsets = pd.read_excel('data/emerald_legacy_changedex.xlsx', sheet_name = 'Level Up Learnsets', header = 0, index_col = 0)


# Take only the last 4 moves
move_columns = all_learnsets.columns[1:]

# Step 2: For each row, get the last 4 non-NaN moves
last_4_moves = (
    all_learnsets[move_columns]
    .apply(lambda row: row.dropna().tail(4).tolist(), axis=1)  # Get last 4 moves per row
    .apply(pd.Series)  # Convert lists to columns
    .rename(columns=lambda x: f'Move {x + 1}')  # Rename columns to 'Move 1' to 'Move 4'
)

learnsets = pd.concat([all_learnsets[['Pokemon']], last_4_moves], axis=1)

learnsets[learnsets.columns[1:]] = learnsets[learnsets.columns[1:]].apply(lambda col: col.str.split(', ').str[-1])

In [105]:
learnsets.sample(10)

,Pokemon,Move 1,Move 2,Move 3,Move 4
Index,,,,,
337.0,Dusclops,Will O Wisp,Snatch,Mean Look,Future Sight
65.0,Alakazam,Future Sight,Calm Mind,Psychic,Trick
165.0,Ledyba,Signal Beam,Giga Drain,Safeguard,Double Edge
289.0,Wailord,Rest,Water Spout,Amnesia,Hydro Pump
222.0,Corsola,Spike Cannon,Rock Blast,Mirror Coat,Ancient Power
179.0,Mareep,Thunder Wave,Cotton Spore,Light Screen,Thunder
252.0,Treecko,Agility,Slam,Detect,Giga Drain
283.0,Spinda,Thrash,Outrage,Flail,Double Edge
356.0,Relicanth,Sleep Talk,Double Edge,Rock Slide,Hydro Pump


## Functions

In [218]:
def calculate_damage(att_pokemon, def_pokemon, move):
    """
    Calculate damage based on the Pokémon type effectiveness chart.
    """
    level = 50
    random = np.random.uniform(0.85, 1)

    power = move['Power']
    attack = att_pokemon['Attack'] if move['Category'] == 'Physical' else att_pokemon['Sp. Atk']
    defense = def_pokemon['Defense'] if move['Category'] == 'Physical' else def_pokemon['Sp. Def']
    stab = 1.5 if att_pokemon['Type 1'] == move['Type'] or att_pokemon['Type 2'] == move['Type'] else 1
    effectiveness = matchups.loc[move['Type'], def_pokemon['Type 1']] * matchups.loc[move['Type'], def_pokemon['Type 2']]

    # Calculate damage
    damage = ((((2*level/5) + 2)*power*attack/(50*defense)) + 2)*stab*effectiveness*random
    damage = damage if math.isnan(power) else int(damage)
    return damage

## Prototype (simulate move usage)

In [219]:
## simulate move in a battle between Charizard and Venusaur

charizard = all_pokemon[all_pokemon['Name'] == 'Charizard'][['Type 1', 'Type 2', 'Attack', 'Sp. Atk', 'Defense', 'Sp. Def']].iloc[0]
venusaur = all_pokemon[all_pokemon['Name'] == 'Venusaur'][['Type 1', 'Type 2', 'Attack', 'Sp. Atk', 'Defense', 'Sp. Def']].iloc[0]

charizard_moves = learnsets[learnsets['Pokemon'] == 'Charizard'].iloc[0, 1:].values
venusaur_moves = learnsets[learnsets['Pokemon'] == 'Venusaur'].iloc[0, 1:].values

In [220]:
moves[moves['Name'] == 'Dragon Rage'][['Name', 'Type', 'Accuracy', 'Power', 'Category']].iloc[0, 1:]

Type         Dragon
Accuracy      100.0
Power           NaN
Category    Special
Name: 81, dtype: object

In [223]:
for i in range(5):
    move_name = charizard_moves[random.randint(0, 3)]
    move = moves[moves['Name'] == move_name][['Name', 'Type', 'Accuracy', 'Power', 'Category']].iloc[0, 1:]
    dmg = calculate_damage(charizard, venusaur, move)    
    print(f"Charizard uses {move_name} on Venusaur \nDamage: {dmg:.1f} \n")


Charizard uses Blast Burn on Venusaur 
Damage: 191.0 

Charizard uses Body Slam on Venusaur 
Damage: 37.0 

Charizard uses Blast Burn on Venusaur 
Damage: 217.0 

Charizard uses Blast Burn on Venusaur 
Damage: 201.0 

Charizard uses Body Slam on Venusaur 
Damage: 34.0 



In [224]:
for i in range(5):
    move_name = venusaur_moves[random.randint(0, 3)]
    move = moves[moves['Name'] == move_name][['Name', 'Type', 'Accuracy', 'Power', 'Category']].iloc[0, 1:]
    dmg = calculate_damage(venusaur, charizard, move)    
    print(f"Venusaur uses {move_name} on Charizard \nDamage: {dmg:.1f} \n")


Venusaur uses Frenzy Plant on Charizard 
Damage: 25.0 

Venusaur uses Body Slam on Charizard 
Damage: 41.0 

Venusaur uses Frenzy Plant on Charizard 
Damage: 26.0 

Venusaur uses Frenzy Plant on Charizard 
Damage: 26.0 

Venusaur uses Solar Beam on Charizard 
Damage: 21.0 



### Next steps

To be developed: 

- Simulate a single battle
- Create AI to choose which move each pokemon will use in each turn
- Simulate status, stats changes, secondary effects (maybe recoil)
- Add complexity to moves (attack missed, crit., flinch, etc.)

And some more. 
